In [1]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
model.to('cuda')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [1]:
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Databricks MLflow 추적 서버 URI 설정
databricks_uri = "databricks"  # Databricks 워크스페이스의 MLflow URI
mlflow.set_tracking_uri(databricks_uri)

# MLflow 추적 서버 URI 설정 (Azure Databricks 내에서 자동으로 설정될 수 있음)
# mlflow.set_tracking_uri('<Azure Databricks MLflow URI>')

# MLflow 실험 설정
mlflow.set_experiment('/Users/20182562@oasis.inje.ac.kr/IrisClassification')

# 실험 시작
with mlflow.start_run():
    # 데이터 로드 및 분할
    iris = load_iris()
    X = iris.data
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 생성 및 훈련
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)

    # 예측 및 정확도 계산
    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)

    # 결과 출력
    print(f"정확도: {accuracy}")

    # 파라미터, 메트릭스, 모델을 MLflow에 기록
    mlflow.log_param("test_size", 0.2)
    mlflow.log_param("random_state", 42)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.sklearn.log_model(clf, "model")



2023/12/27 21:53:42 INFO mlflow.tracking.fluent: Experiment with name '/Users/20182562@oasis.inje.ac.kr/IrisClassification' does not exist. Creating a new experiment.


정확도: 1.0


2023/12/27 21:53:44 WARNING mlflow.models.model: Model logged without a signature. Signatures will be required for upcoming model registry features as they validate model inputs and denote the expected schema of model outputs. Please visit https://www.mlflow.org/docs/2.9.2/models.html#set-signature-on-logged-model for instructions on setting a model signature on your logged model.
/home/stonedev/anaconda3/envs/dev_torch/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import mlflow
import mlflow.pytorch
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

# MLflow 추적 서버 URI 설정 (Azure Databricks 내에서 자동으로 설정될 수 있음)
databricks_uri = "databricks"  # Databricks 워크스페이스의 MLflow URI
mlflow.set_tracking_uri(databricks_uri)

# MLflow 실험 설정
mlflow.set_experiment('/Users/20182562@oasis.inje.ac.kr/YOLOv8CustomTraining')

# 실험 시작
with mlflow.start_run():
    # YOLOv8 커스텀 데이터 학습 설정
    config_path = '<path_to_your_custom_config>'  # YOLOv8 커스텀 설정 파일 경로
    data_path = '<path_to_your_custom_data>'     # 데이터셋 경로
    epochs = 100                                # 에포크 수
    batch_size = 32                             # 배치 사이즈

    # 학습 시작
    results = train(config_path, data_path, epochs=epochs, batch_size=batch_size)

    # 학습 파라미터 기록
    mlflow.log_param("config_path", config_path)
    mlflow.log_param("data_path", data_path)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)

    # 학습 메트릭스 기록 (예시: 손실, 정확도 등)
    # results 객체에서 필요한 메트릭스 정보를 추출하여 기록합니다.
    # 이 부분은 YOLOv8의 train 함수 반환값에 따라 달라질 수 있습니다.
    mlflow.log_metrics({"loss": results.loss, "accuracy": results.accuracy})

    # 학습된 모델 기록
    mlflow.pytorch.log_model(results.model, "model")


In [2]:
import mlflow
import re,os
# Databricks MLflow 추적 서버 URI 설정
databricks_uri = "databricks"  # Databricks 워크스페이스의 MLflow URI
# mlflow.set_tracking_uri(databricks_uri)
os.environ['MLFLOW_TRACKING_URI'] = databricks_uri
# # Set the experiment path
mlflow.set_experiment("/Users/20182562@oasis.inje.ac.kr/Yolov")

def on_fit_epoch_end(trainer):
    if mlflow:
        metrics_dict = {f"{re.sub('[()]', '', k)}": float(v) for k, v in trainer.metrics.items()}
        mlflow.log_metrics(metrics=metrics_dict, step=trainer.epoch)

# 실험 세션 생성
try: 
     with mlflow.start_run():
          model.add_callback("on_fit_epoch_end",on_fit_epoch_end)
          result = model.train(data='../BP_OB_Model/Food_OD.yaml', epochs=100, patience=30, batch=32, imgsz=416)
          mlflow.pytorch.log_model(result.model, "model")
	
except mlflow.exceptions.MlflowException as e:
    print(f"An Mlflow error occurred: {str(e)}")

2023/12/27 22:17:28 INFO mlflow.tracking.fluent: Experiment with name '/Users/20182562@oasis.inje.ac.kr/Yolov' does not exist. Creating a new experiment.


New https://pypi.org/project/ultralytics/8.0.230 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../BP_OB_Model/Food_OD.yaml, epochs=100, time=None, patience=30, batch=32, imgsz=416, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /data/BP/BP_OB_Model/Train.cache... 41065 images, 0 backgrounds, 37 corrupt: 100%|██████████| 41101/41101 [00:00<?, ?it/s]

train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_000_0097.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_000_0233.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0181.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0286.jpg: ignoring corrupt image/label: invalid image format GIF
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0355.jpg: ignoring corrupt image/label: invalid image format GIF
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0362.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_002_0586.JPG: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_002_0879.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_004_0412.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_004_0419.jpg: corrupt JPEG restored and saved
trai


val: Scanning /data/BP/BP_OB_Model/Val.cache... 10194 images, 0 backgrounds, 6 corrupt: 100%|██████████| 10199/10199 [00:00<?, ?it/s]

...

                   all      10193      10193      0.487      0.436      0.419      0.275

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      9/100      2.59G      1.176      1.975      1.349         80        416:   3%|▎         | 40/1284 [00:04<02:20,  8.87it/s]libpng warning: iCCP: known incorrect sRGB profile
      9/100      2.59G      1.175      1.986      1.351         86        416:   4%|▍         | 50/1284 [00:05<02:18,  8.90it/s]libpng warning: iCCP: known incorrect sRGB profile
      9/100      2.59G      1.172      1.997      1.354         89        416:   6%|▋         | 81/1284 [00:09<02:13,  9.04it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
      9/100      2.59G      1.166      1.987      1.353         75        416:  10%|▉         | 124/1284 [00:13<02:10,  8.91it/s]libpng warning: iCCP: known incorrect sRGB profile
      9/100      2.59G      1.168      1.981      1.354         84        416:  12%|█▏        | 157/1284 [00:17<02:04,  9.05it/s]libpng warning: iCCP: known incorrect sRGB profile
      9/100      2.59G      1.171      1.978      1.

                   all      10193      10193      0.526      0.438      0.434      0.284

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     10/100      2.59G      1.123      1.952      1.307         86        416:   1%|          | 13/1284 [00:01<02:22,  8.92it/s]libpng warning: iCCP: known incorrect sRGB profile
     10/100      2.59G      1.135      1.916      1.326         88        416:   3%|▎         | 34/1284 [00:03<02:20,  8.89it/s]libpng warning: iCCP: known incorrect sRGB profile
     10/100      2.59G      1.141      1.912      1.334         95        416:   5%|▍         | 63/1284 [00:07<02:15,  9.00it/s]libpng warning: iCCP: known incorrect sRGB profile
     10/100      2.59G      1.162      1.926      1.346         72        416:  15%|█▍        | 187/1284 [00:21<02:01,  9.01it/s]libpng warning: iCCP: known incorrect sRGB profile
     10/100      2.59G      1.164      1.922      1.347         84        416:  16%|█▌        | 206/1284 [00:23<02:01,  8.87it/s]libpng warning: iCCP: known incorrect sRGB profile
     10/100      2.59G      1.165       1.93      1.348         87        416:  19%|█▊        | 239/128

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 3.
Original Traceback (most recent call last):
  File "/home/stonedev/anaconda3/envs/dev_torch/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/stonedev/anaconda3/envs/dev_torch/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/stonedev/anaconda3/envs/dev_torch/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/home/stonedev/anaconda3/envs/dev_torch/lib/python3.11/site-packages/ultralytics/data/base.py", line 248, in __getitem__
    return self.transforms(self.get_image_and_label(index)